This notebook is my practise to do EDA (exploratory data analysis), the first step in data science.

I used to be interested on daily covid data when covid hit Indonesia back in March and April. It had been a while, so let's have a look at them now

Data will be scrapped from:
* https://github.com/CSSEGISandData/COVID-19
* Our World in Data (for Indonesia): https://github.com/owid/covid-19-data/tree/master/public/data
* Covid data for Indonesia from KawalCOVID: http://sinta.ristekbrin.go.id/covid/datasets
* GEOjson for Indonesia: https://bitbucket.org/rifani/geojson-political-indonesia/src/master/

Summary of this notebook:
* Download Covid19 data for worldwide and Indonesia from several sources
* Download GEOjson map for Indonesia
* Worldwide: summary of latest data, worldwide map, death vs confirmed comparison
* Indonesia: summary of latest data, summary plots (total cases, daily cases, positive rate and mortality rate) and other random stats that I am interested in

New skills I picked up and applied on this notebook:
* Using Plotly Express
* Extracting data from Google Sheet API
* Cleaning data. The spreadsheet is messy. Table are stacked on other tables in the same spreadsheet tab
* Extracted data is string. Not sure if there is a way to extract in a numeric format instead of converting it to float manually. For next time, maybe there is a way to just download from Google Sheet automatically and just pd.read_csv()
* Working with GEOjson data format and plotting an interactive map


# Covid19 in Indonesia

# Import necessary python libraries

In [1]:
#collapse
# download python libraries
from datetime import datetime, timedelta
import os
import glob
import wget
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import json
import plotly.express as px
import plotly.graph_objs as go

# for offline ploting
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

# Import data

In [2]:
#collapse
# Download data from Github (daily)
os.chdir("C:/Users/Riyan Aditya/Desktop/ML_learning/Project4_EDA_Covid_Indo/datasets")

os.remove('time_series_covid19_confirmed_global.csv')
os.remove('time_series_covid19_deaths_global.csv')
os.remove('time_series_covid19_recovered_global.csv')

# urls of the files
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv']

# download files
for url in urls:
    filename = wget.download(url)

100% [............................................................................] 264014 / 264014

# Clean & preprocess data